In [1]:
from read_datasetBreakfast import load_data, read_mapping_dict
import os
import numpy as np


COMP_PATH = ''

''' 
training to load train set
test to load test set
'''
split = 'training'
#split = 'test'
train_split =  os.path.join(COMP_PATH, 'splits/dev_train.split1.bundle') #Train Split
test_split  =  os.path.join(COMP_PATH, 'splits/test.split1.bundle') #Test Split
GT_folder   =  os.path.join(COMP_PATH, 'groundTruth/') #Ground Truth Labels for each training video 
DATA_folder =  os.path.join(COMP_PATH, 'Data/') #Frame I3D features for all videos
mapping_loc =  os.path.join(COMP_PATH, 'splits/mapping_bf.txt') 

actions_dict = read_mapping_dict(mapping_loc)

data_feat, data_labels = load_data( train_split, actions_dict, GT_folder, DATA_folder, datatype = split) #Get features and labels

Finish Load the Training data and labels!!!


In [6]:
from Dataset.VideoDataset import VideoDataset
import torch
import torch.utils.data as tud
from Models.LSTM import LSTM_Model
import torch.nn.functional as F
from sklearn.metrics import accuracy_score

Batch_Size = 10
epochs = 10

dataset = VideoDataset(data_feat, data_labels)
dataloader = tud.DataLoader(dataset, shuffle=True)

model = LSTM_Model()
learning_rate = 1e-3
log_interval = 10

def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()
    
    losses = []
    scores = []
    for batch_idx, (in_feature, label) in enumerate(train_loader):
        in_feature = in_feature.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()
        output = model(in_feature)[0]
        loss = F.nll_loss(output, label[0][0])
        losses.append(loss.item())
        
        label_predict = torch.max(output, 1)[1]
        step_score = accuracy_score(label.cpu().data.squeeze().numpy(), label_predict.cpu().data.squeeze().numpy())
        scores.append(step_score)
        
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Accu: {:.2f}%'.format(
                epoch + 1, batch_idx, len(train_loader.dataset), 100. * (batch_idx + 1) / len(train_loader), loss.item(), 100 * step_score))
        
    return losses, scores

In [7]:
cuda_avail = torch.cuda.is_available()
device = torch.device("cuda" if cuda_avail else "cpu")

lstm = LSTM_Model().double().to(device)
optimizer = torch.optim.Adam(list(lstm.parameters()), lr=learning_rate)

train_losses, train_scores = train(log_interval, lstm, device, dataloader, optimizer, 0)

Train Epoch: 1 [0/174 (1%)]	Loss: 7.006767, Accu: 0.91%
Train Epoch: 1 [10/174 (6%)]	Loss: 8.280340, Accu: 5.16%
Train Epoch: 1 [20/174 (12%)]	Loss: 8.240985, Accu: 3.64%


KeyboardInterrupt: 